<a href="https://colab.research.google.com/github/FGalvao77/Comparando-a-performance---XGBoost-VS-LightGBM/blob/main/XGBoost_VS_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Comparando a performance - XGBoost VS LightGBM**

> Iremos realizar a comparação entre os algoritmos `XGBoost` e `LightGBM` e, avaliar suas performances quanto algumas métricas e tempo de execução do algoritmo.

**Documentação:**

- [XGBoost](https://xgboost.readthedocs.io/en/stable/)
- [LightGBM](https://lightgbm.readthedocs.io/en/latest/)

## **1. Instalando e importando as bibliotecas**

In [1]:
# importando a biblioteca para ignorar "erros"
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# instalando o "XGBoot"
!pip install xgboost

In [3]:
# instalando o "LightGBM"
!pip install lightgbm

In [4]:
# importando as biblioetcas
import pandas as pd
import numpy as np
import lightgbm as lgb  
import xgboost as xgb 

**Carregando e explorando o conjunto de dados**

In [5]:
# carregando o conjunto de dados para dentro do ambiente do google colab
!wget 'https://raw.githubusercontent.com/llSourcell/Best-Programming-Languages-for-Machine-Learning/master/adults.txt'

--2021-11-26 00:24:31--  https://raw.githubusercontent.com/llSourcell/Best-Programming-Languages-for-Machine-Learning/master/adults.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3518613 (3.4M) [text/plain]
Saving to: ‘adults.txt.4’

adults.txt.4        100%[===================>]   3.36M  --.-KB/s    in 0.02s   

2021-11-26 00:24:31 (157 MB/s) - ‘adults.txt.4’ saved [3518613/3518613]



In [6]:
# realizando a leitura do conjunto de dados e instanciando na variável "df"
df = pd.read_csv('/content/adults.txt', sep=',')

# visualizando o objeto criado
df

,age,workclass,final_weight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [7]:
# tipo do objeto criado
type(df)

pandas.core.frame.DataFrame

In [8]:
# nome das colunas
df.columns

Index(['age', 'workclass', 'final_weight', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'salary'],
      dtype='object')

In [9]:
# informações gerais
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   final_weight    32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


**O nosso desafio é prever dados alguns atributos qual o salário de um colaborador, portanto um problema de _classificação_**

Vamos explorar a nosso variável alvo, nesse caso o `salary` entender as condições da mesma.

In [10]:
# visualizando as instâncias únicas da variável alvo
df['salary'].unique()

array(['<=50K', '>50K'], dtype=object)

In [11]:
# contabilizando as instâncias únicas da variável alvo
df['salary'].value_counts()

<=50K    24720
>50K      7841
Name: salary, dtype: int64

> Perceba que, a variável alvo possui dado do tipo _object_, ou seja, caracteres (strings). E sua relação é categórica:    
- salário maior e/ou igual a 50 mil (`<=50K`) e,
- salário menor que 50 mil (`>50K`).

É necessário realizarmos um tratamento em nossa variável alvo, transformando-a para do tipo numérica.

## **2. Pré-processamento dos dados**

In [12]:
# importando a biblioteca para tratamento da variável alvo
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [13]:
# instanciando a função no objeto "lb"
lb = LabelEncoder()

# aplicando a função na variável alvo
lb.fit(df['salary'])

LabelEncoder()

> O `LabelEncoder` transformará as instâncias em classes.

In [14]:
# visualizando as classes criadas
lb.classes_

array(['<=50K', '>50K'], dtype=object)

In [15]:
# contabilizando as instâncias presentes
df['salary'].value_counts()

<=50K    24720
>50K      7841
Name: salary, dtype: int64

In [16]:
# aplicando a transformação das classes para tipo numérico
# para isso, utilizamos a função ".transform()" e juntamente com a função do pandas ".Series()"
df['salary'] = pd.Series(lb.transform(df['salary']))

In [17]:
# visualizando e contabilizando a transformação realizada
df['salary'].value_counts()

0    24720
1     7841
Name: salary, dtype: int64

In [18]:
# informações gerais
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   final_weight    32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  salary          32561 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 3.7+ MB


In [19]:
# visualizando as instâncias únicas dos atributos
df.nunique()

age                  73
workclass             9
final_weight      21648
education            16
education_num        16
marital_status        7
occupation           15
relationship          6
race                  5
sex                   2
capital_gain        119
capital_loss         92
hours_per_week       94
native_country       42
salary                2
dtype: int64

In [20]:
# visualizando as instâncias únicas do atributo "workclass"
df['workclass'].unique()

array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
       'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'],
      dtype=object)

In [21]:
# contabilizando a frequência absoluta (%) das instãncias do atributo "workclass"
df['workclass'].value_counts(normalize=True) * 100

Private             69.703019
Self-emp-not-inc     7.803814
Local-gov            6.427935
?                    5.638647
State-gov            3.986364
Self-emp-inc         3.427413
Federal-gov          2.948312
Without-pay          0.042996
Never-worked         0.021498
Name: workclass, dtype: float64

> Perceba que, temos uma instância atribuida de "`?`" e, portanto devemos tratar desse problema.

In [22]:
# visualizando a instância mais frequente (moda)
moda = df['workclass'].value_counts().idxmax()
moda

'Private'

In [23]:
# realizando a substituição do caracter "?" pelo valor da moda, nesse caso "Private"
df['workclass'] = df['workclass'].apply(lambda x: x.replace('?','Private'))
df['workclass'].unique()

array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
       'Local-gov', 'Self-emp-inc', 'Without-pay', 'Never-worked'],
      dtype=object)

**Agora temos que realizar um tratamento para as demais _variáveis categóricas_.**

In [24]:
# instanciando as variáveis categóricas
vars_cat = [col for col in df if df[col].dtype.name == 'object']

# visualizando as variáveis categóricas
df[vars_cat]

,workclass,education,marital_status,occupation,relationship,race,sex,native_country
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba
...,...,...,...,...,...,...,...,...
32556,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States
32557,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States
32558,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States
32559,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,United-States


> Para essa etapa, iremos aplicar a função do pandas `.get_dummies` que irar transformar as variáveis categóricas.

In [25]:
# instanciando cada atributo e realizando a transformação necessária
ohe_workclass = pd.get_dummies(df[vars_cat[0]])
ohe_education = pd.get_dummies(df[vars_cat[1]])
ohe_marital_status = pd.get_dummies(df[vars_cat[2]])
ohe_occupation = pd.get_dummies(df[vars_cat[3]])
ohe_relationship = pd.get_dummies(df[vars_cat[4]])
ohe_race = pd.get_dummies(df[vars_cat[5]])
ohe_sex = pd.get_dummies(df[vars_cat[6]])
ohe_native_country = pd.get_dummies(df[vars_cat[7]])

In [26]:
# visualizando o primeiro objeto e as transformações realizadas
ohe_workclass

,Federal-gov,Local-gov,Never-worked,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay
0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...
32556,0,0,0,1,0,0,0,0
32557,0,0,0,1,0,0,0,0
32558,0,0,0,1,0,0,0,0
32559,0,0,0,1,0,0,0,0


> Foi criado um atributo relacionado com cada uma das instâncias presentes na coluna.

In [27]:
# concatenado as transformações realizadas juntamente com o "df" e instanciando em um novo objeto "df2"
df2 = pd.concat([df, ohe_workclass, ohe_education, ohe_marital_status, ohe_occupation, ohe_relationship, 
               ohe_race, ohe_sex, ohe_native_country], axis=1)

In [28]:
# removendo as variáveis que não faz mais sentido, já que as mesmas foram tratadas
df2.drop(['education', 'marital_status', 'native_country', 'occupation', 
                'race', 'relationship', 'sex', 'workclass'], 
               axis=1, inplace=True)

In [29]:
# informações gerais
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Columns: 108 entries, age to Yugoslavia
dtypes: int64(7), uint8(101)
memory usage: 4.9 MB


In [30]:
# visualizando os atributos presentes no objeto "df2" até a coluna 50a 
df2.columns[:50]

Index(['age', 'final_weight', 'education_num', 'capital_gain', 'capital_loss',
       'hours_per_week', 'salary', 'Federal-gov', 'Local-gov', 'Never-worked',
       'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov',
       'Without-pay', '10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th',
       '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad',
       'Masters', 'Preschool', 'Prof-school', 'Some-college', 'Divorced',
       'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent',
       'Never-married', 'Separated', 'Widowed', '?', 'Adm-clerical',
       'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing',
       'Handlers-cleaners', 'Machine-op-inspct', 'Other-service',
       'Priv-house-serv', 'Prof-specialty', 'Protective-serv'],
      dtype='object')

In [31]:
# visualizando os demais atributos presentes da coluna 50a em diante
df2.columns[50:]

Index(['Sales', 'Tech-support', 'Transport-moving', 'Husband', 'Not-in-family',
       'Other-relative', 'Own-child', 'Unmarried', 'Wife',
       'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White',
       'Female', 'Male', '?', 'Cambodia', 'Canada', 'China', 'Columbia',
       'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England',
       'France', 'Germany', 'Greece', 'Guatemala', 'Haiti',
       'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran',
       'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua',
       'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland',
       'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand',
       'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'],
      dtype='object')

In [32]:
# visualizando novamente a variável alvo
df2['salary']

0        0
1        0
2        0
3        0
4        0
        ..
32556    0
32557    1
32558    0
32559    0
32560    1
Name: salary, Length: 32561, dtype: int64

In [33]:
# contabilizando a presença de dados duplicados
df2.duplicated().sum()

24

In [34]:
# removendo dados duplicados
df2.drop_duplicates(keep=False, inplace=True)

In [35]:
# contabilizando a presença de dados duplicados apoś o tratamento
df2.duplicated().sum()

0

In [36]:
# contabilizando a quantidade de atributos presentes no "df2"
_, i = np.unique(df2.columns, return_index=True)

df2 = df2.iloc[:, i]

In [37]:
# informações gerais
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32514 entries, 0 to 32560
Columns: 107 entries, 10th to salary
dtypes: int64(7), uint8(100)
memory usage: 5.1 MB


In [38]:
# realizando o particionamento dos dados para treino/validação e teste final
data = df2.sample(frac=0.7, random_state=42)
val_data = df2.drop(data.index)

# resetando os índices
data.reset_index(drop=True, inplace=True)
val_data.reset_index(drop=True, inplace=True)

# visualizando a dimensão do particionamento dos dados
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(val_data.shape))

Data for Modeling: (22760, 107)
Unseen Data For Predictions: (9754, 107)


In [39]:
# instanciando as variáveis explicativas (X) e a resposta (y)
X = data.drop('salary', axis=1)
y = data['salary']

In [40]:
# visualizando a dimensão do objeto
X.shape

(22760, 106)

In [41]:
# contabilizando a presença de valores "NaN" na varável "y" (alvo)
y.isna().sum()

0

> Não há valores `NaN`.

Mas se no caso houvesse poderíamos utilizar alguma estratégia para tratar o problema, como:    
- remover as linhas com valores do tipo `NaN'.
 - esse tipo de ação pode gerar a perda considerável de dados.
- e preencher com algum valor presente no atributo:    
 - média;
 - mediana e,
 - moda.

In [42]:
# aplicando o preenchimento com a moda
y.fillna(y.mode()[0], inplace=True)

In [43]:
# importando a biblioteca para realizar a partição do conjunto de dados
from sklearn.model_selection import train_test_split

In [44]:
# particionando os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                      test_size=0.3,
                                                      random_state=42)

In [45]:
# visualizando a dimensão do particionamento dos dados
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15932, 106), (6828, 106), (15932,), (6828,))

## **3. Modelagem e avaliação do modelo**

### **XGBoost**




Antes de aplicar o algoritmo `XGBoost` é necessário realizar uma transformação nas bases de dados, pois o modelo trabalha com matrizes.

Para isso, utilizamos a função `.DMatrix()`.

In [46]:
# instanciando o objeto "df_train", para isso utilizamos a função ".DMatrix" do XGBoost
# e passando como argumentos os dados de treino "df_train" e os seus rótulos "y_train"
df_train = xgb.DMatrix(X_train, label=y_train)

# instanciando o objeto "df_test", para isso utilizamos a função ".DMatrix" do XGBoost
df_test = xgb.DMatrix(X_test)

In [47]:
# instanciando um dicionário com os parâmetros do primeiro modelo
params_xgb = {'max_depth': 5,                               # profundidade máxima da árvore
              'eta': 1,                                     # taxa de aprendizagem
              'silent': 1,                                  # 
              'objective': 'binary:logistic',               # objetivo é binário - duas classes | 0 e 1
              'eval_metric': 'auc', 'learning_rate': 0.5    #
            }   

In [48]:
# instanciando um dicionário com os parâmetros do segundo modelo
params_xgb2 = {
    'n_estimators': 1300,
    'max_depth': 6,
    'min_child_weight': 4,
    'subsample': 0.9,
    'colsample_bynode': 0.5,
    'random_state': 42, 
    'num_parallel_tree': 3,
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eval_metric': 'auc', 'learning_rate': 0.5 
}                            

In [49]:
# definindo o número de treinamento
num_round = 150

# importando a função para contabilizar o tempo de treinbamento do modelo
from datetime import datetime
start = datetime.now()

In [50]:
# treinando o primeiro modelo
model_xgb = xgb.train(params_xgb, df_train, num_round)
stop = datetime.now()

In [51]:
# visualizando o seu tempo de execução
execution_time_xgb = stop - start
execution_time_xgb

datetime.timedelta(seconds=7, microseconds=17011)

In [52]:
# realizando as predições
y_pred_xgb = model_xgb.predict(df_test)

# tamanho das predições realizadas
y_pred_xgb.size

6828

In [53]:
# visualizando as 10 primeiras predições
y_pred_xgb[:10]

array([4.9780652e-02, 7.6236590e-03, 1.3620844e-04, 8.8696692e-07,
       1.1847151e-02, 4.4916971e-03, 2.0786809e-02, 5.0980458e-04,
       5.7433289e-03, 2.6179215e-02], dtype=float32)

> O modelo gera as probabilidades das instâncias pertence a classe 0 ou 1.

In [54]:
# realizando um "for" para definir o "corte" das probabilidades realizadas pelo modelo
# e instanciando as classes
for i in range(0, len(y_pred_xgb)):
    if y_pred_xgb[i] >= 0.5:
        y_pred_xgb[i] = 1
    else:
        y_pred_xgb[i] = 0

In [55]:
# importando a biblioteca para avaliação do modelo
from sklearn.metrics import accuracy_score

In [56]:
# realizando avaliação do modelo
acc_model_xgb = accuracy_score(y_pred_xgb, y_test)
acc_model_xgb

0.8615992970123023

**Aplicando o segundo modelo do `XGBoost` e _tunando_ alguns parâmetros.**

In [57]:
# treinando o segundo modelo
model_xgb2 = xgb.train(params_xgb2, df_train, num_round)
stop = datetime.now()

In [58]:
# visualizando o seu tempo de execução
execution_time_xgb2 = stop - start
execution_time_xgb2

datetime.timedelta(seconds=21, microseconds=174890)

In [59]:
# realizando as predições
y_pred_xgb2 = model_xgb2.predict(df_test)

# tamanho das predições realizadas
y_pred_xgb2.size

6828

In [60]:
# visualizando as 10 primeiras predições
y_pred_xgb2[:10]

array([1.0843548e-02, 8.9237355e-03, 1.8119768e-04, 1.5445412e-05,
       1.9114583e-03, 1.3707524e-02, 1.6958008e-02, 7.0886203e-04,
       1.0937657e-02, 8.5898601e-02], dtype=float32)

In [61]:
# realizando um "for" para definir o "corte" das probabilidades realizadas pelo modelo
# e instanciando as classes
for i in range(0, len(y_pred_xgb2)):
    if y_pred_xgb2[i] >= 0.5:
        y_pred_xgb2[i] = 1
    else:
        y_pred_xgb2[i] = 0

In [62]:
# realizando avaliação do modelo
acc_model_xgb2 = accuracy_score(y_pred_xgb2, y_test)
acc_model_xgb2

0.8613063854715876

### **LightGBM**

Já o algoritmo `LightGBM` trabalha com objeto do tipo "_Dataset_", para isso utilizamos a transformação dos dados com a função `.Dataset()`.

In [63]:
# instanciando o objeto "train_data", para isso utilizamos a função ".Dataset" do LightGBM
# e passando como argumentos os dados de treino "X_train" e os seus rótulos "y_train"
train_data = lgb.Dataset(X_train, label=y_train)

In [64]:
# instanciando os parãmetros do modelo
params_lgb = {
    'num_leaves': 150,
    'objective': 'binary',
    'max_depth': 6,
    'learning_rate': 0.5,
    'max_bin': 200
}

In [65]:
# definindo as métricas de avaliação do modelo
params_lgb['metric'] = ['auc', 'binary_logloss']

In [66]:
# definindo o número de treinamento
num_round = 150

# treinando o modelo e contabilizando o seu tempo de excução
start = datetime.now()
model_lgb = lgb.train(params_lgb, train_data, num_round)
stop = datetime.now()

# visualizando o tempo de excução
execution_time_lgb = stop - start
execution_time_lgb

datetime.timedelta(microseconds=461962)

In [67]:
# realizando as predições
y_pred_lgb = model_lgb.predict(X_test)
y_pred_lgb.size

6828

In [68]:
# visualizando as 10 primeiras predições
y_pred_lgb[:10]

array([1.02582132e-02, 3.80888220e-03, 1.73859161e-06, 4.11561875e-10,
       7.53310803e-04, 5.03548931e-03, 3.96304238e-03, 6.48394063e-06,
       1.71235019e-02, 3.66456324e-02])

In [69]:
# realizando um "for" para definir o "corte" das probabilidades realizadas pelo modelo
# e instanciando as classes
for i in range(0, len(y_pred_lgb)):
    if y_pred_lgb[i] >= 0.5:
        y_pred_lgb[i] = 1
    else:
        y_pred_lgb[i] = 0

In [70]:
# avaliando o modelo
acc_model_lgb = accuracy_score(y_pred_lgb, y_test)
acc_model_lgb

0.8517867603983597

**Aplicando o segundo modelo do `LightGBM` e _tunando_ alguns parâmetros.**

In [71]:
# segundo modelo do "LigthGBM"
train_data = lgb.Dataset(X_train, y_train)
test_data = lgb.Dataset(X_test, y_test, reference=train_data)

In [72]:
params_lgb2 = {
    'boosting_type': 'rf',
    'objective': 'binary',
    'metric': {'auc', 'binary_logloss'},
    'num_leaves': 150,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'max_bin': 200,
    'verbose': 0
}

In [73]:
# visualizando os parâmetros do treinamento do modelo
params_lgb2

{'bagging_fraction': 0.8,
 'bagging_freq': 5,
 'boosting_type': 'rf',
 'feature_fraction': 0.9,
 'learning_rate': 0.05,
 'max_bin': 200,
 'metric': {'auc', 'binary_logloss'},
 'num_leaves': 150,
 'objective': 'binary',
 'verbose': 0}

In [74]:
start = datetime.now()

model_lgb2 = lgb.train(params_lgb2,
                       train_data,
                       num_boost_round=150,
                       valid_sets=test_data,
                       early_stopping_rounds=10)

stop = datetime.now()

execution_time_lgb2 = stop - start
execution_time_lgb2

[1]	valid_0's auc: 0.896672	valid_0's binary_logloss: 0.354059
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.906771	valid_0's binary_logloss: 0.338443
[3]	valid_0's auc: 0.909213	valid_0's binary_logloss: 0.339516
[4]	valid_0's auc: 0.911084	valid_0's binary_logloss: 0.337141
[5]	valid_0's auc: 0.911108	valid_0's binary_logloss: 0.336774
[6]	valid_0's auc: 0.912458	valid_0's binary_logloss: 0.333654
[7]	valid_0's auc: 0.912387	valid_0's binary_logloss: 0.33334
[8]	valid_0's auc: 0.912767	valid_0's binary_logloss: 0.332781
[9]	valid_0's auc: 0.912901	valid_0's binary_logloss: 0.332634
[10]	valid_0's auc: 0.912533	valid_0's binary_logloss: 0.332977
[11]	valid_0's auc: 0.913042	valid_0's binary_logloss: 0.33243
[12]	valid_0's auc: 0.913178	valid_0's binary_logloss: 0.33245
[13]	valid_0's auc: 0.913081	valid_0's binary_logloss: 0.332605
[14]	valid_0's auc: 0.913202	valid_0's binary_logloss: 0.332137
[15]	valid_0's auc: 0.912988	valid_0's binary_logloss:

datetime.timedelta(microseconds=629584)

In [75]:
model_lgb2.save_model('model.txt')

In [76]:
y_pred_lgb2 = model_lgb2.predict(X_test, num_iteration=model_lgb2.best_iteration)

In [77]:
for i in range(0, len(y_pred_lgb2)):
    if y_pred_lgb2[i] >= 0.5:
        y_pred_lgb2[i] = 1
    else:
        y_pred_lgb2[i] = 0

In [78]:
# avaliando o modelo
acc_model_lgb2 = accuracy_score(y_pred_lgb2, y_test)
acc_model_lgb2

0.859402460456942

**Avaliando os modelos**

In [79]:
# importando a função para avaliação da acurácia da curva roc
from sklearn.metrics import roc_auc_score

In [80]:
# visualizando a acurácia da curva roc do modelo "model_xgb"
auc_xgb1 = roc_auc_score(y_pred_xgb, y_test)
auc_xgb1

0.8165609919364478

In [81]:
# visualizando a acurácia da curva roc do modelo "model_xgb2"
auc_xgb2 = roc_auc_score(y_pred_xgb2, y_test)
auc_xgb2

0.8155764521194301

In [82]:
# visualizando a acurácia da curva roc do modelo "model_lgb"
auc_lgb = roc_auc_score(y_pred_lgb, y_test)
auc_lgb

0.7997189498443203

In [83]:
# visualizando a acurácia da curva roc do modelo "model_lgb2"
auc_lgb2 = roc_auc_score(y_pred_lgb2, y_test)
auc_lgb2

0.8107612612612612

In [84]:
# instanciando um dicionário com os resultados dos modelos
results_comparison = {
    'accuracy score': (acc_model_xgb, acc_model_xgb2, acc_model_lgb, acc_model_lgb2),
    'auc score': (auc_xgb1, auc_xgb2, auc_lgb, auc_lgb2), 
    'execution time': (execution_time_xgb , execution_time_xgb2, execution_time_lgb, execution_time_lgb2)
}

In [85]:
# criando um dataframe com o dicionário dos resultados dos modelos
df_results = pd.DataFrame(results_comparison, index=['XGBoost_1',
                                                     'XGBoost_2',
                                                     'LightGBM', 
                                                     'LightGBM_2'])

# visualizando o dataframe criado
df_results

,accuracy score,auc score,execution time
XGBoost_1,0.861599,0.816561,0 days 00:00:07.017011
XGBoost_2,0.861306,0.815576,0 days 00:00:21.174890
LightGBM,0.851787,0.799719,0 days 00:00:00.461962
LightGBM_2,0.859402,0.810761,0 days 00:00:00.629584


> Veja que, praticamente as métricas dos modelos ficaram muito próximas uma das outras, pórem o destaque é em relação ao tempo de execução, onde o `LightGBM` foi muito superior ao _XGBoost_.


In [86]:
# salvando o "melhor" modelo, nesse caso o "LightGBM"
model_lgb.save_model('model.txt')

In [87]:
# instanciando o modelo em um arquivo "json"
json_model = model_lgb.dump_model()

In [88]:
# criando um objeto com o modelo salvo
model = lgb.Booster(model_file='model.txt') 

In [89]:
# visualizando o objeto criado
model

In [90]:
# # realizando o download do arquivo
# from google.colab import files 
# files.download('model.txt')